# use after GA (../experiment/0_GA.ipynb) 

In [2]:
import os
import sys

current_dir = os.getcwd()
parent_parent_dir = os.path.dirname(os.path.dirname(current_dir))
src_dir = os.path.join(parent_parent_dir, 'src')
sys.path.append(src_dir)

import ast
import itertools
import random
import time
from statistics import variance

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import umap
from deap import algorithms, base, creator, tools
from ga import not_dup, rand_nodup
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
from scipy.stats import norm
from util import pickle_load, robust_z

In [5]:
for test_num in tqdm(["0701", "0702", "0907", "1001", "1002", "0901", "0902", "0904", "0905"]):
    for lig in ["ago", "anta"]:

        if test_num in ["0701", "0702", "0907", "1001", "1002"]:
            if lig == "ago":
                lig = "!"
            else:
                continue

        seed = 42
        date = 241219
        num = 1

        trys = str(date) + "_" + str(num)

        if lig in ["ago", "anta"]:
            test_name = test_num + "_" + lig
        else:
            test_name = test_num

        for_toxpred_dir = f"../../data/result/{test_name}/GA_{trys}/toxpred_{seed}"

        os.makedirs(for_toxpred_dir, exist_ok=True)


        seed = str(seed)
            
        lookup = pickle_load(f"../../data/processed/{test_name}/for_GA_lookup.pickle")
        cas_tox = pd.read_csv(f"../../data/processed/{test_name}/cas_sev_use.tsv", sep="\t", header=None)
        val = pd.read_csv(f"../../data/processed/{test_name}/validation_pubchem.tsv", sep="\t", header=None)

        ga_result_dir = f"../../data/result/{test_name}/GA_{trys}/"
        ga = pd.read_csv(f"{ga_result_dir}top_5.tsv", sep="\t", header=None)

        random.seed(seed)

        nums = list(range(len(cas_tox)))
        best_pop = set(eval(ga.iloc[-1,0].replace("Individual(", "").replace(")", "")))
        for_eval_set = set(nums) - set(best_pop)
        eval_set = set(random.sample(for_eval_set, len(val)))
        for_test_set = for_eval_set - eval_set
        test_set = set(random.sample(for_test_set, int(len(for_test_set)*0.2)))
        train_set = for_test_set - test_set

        pd.DataFrame([cas_tox.iloc[i] for i in best_pop]).to_csv(f"{for_toxpred_dir}/ga.tsv", sep="\t", header=None, index=False)
        pd.DataFrame([cas_tox.iloc[i] for i in eval_set]).to_csv(f"{for_toxpred_dir}/eval.tsv", header=None, index=False, sep="\t")
        pd.DataFrame([cas_tox.iloc[i] for i in test_set]).to_csv(f"{for_toxpred_dir}/test.tsv", header=None, index=False, sep="\t")
        pd.DataFrame([cas_tox.iloc[i] for i in train_set]).to_csv(f"{for_toxpred_dir}/train.tsv", header=None, index=False, sep="\t")

  0%|          | 0/9 [00:00<?, ?it/s]/tmp/ipykernel_2140424/3740189284.py:40: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  eval_set = set(random.sample(for_eval_set, len(val)))
/tmp/ipykernel_2140424/3740189284.py:42: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  test_set = set(random.sample(for_test_set, int(len(for_test_set)*0.2)))
 11%|█         | 1/9 [01:22<11:01, 82.63s/it]/tmp/ipykernel_2140424/3740189284.py:40: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  eval_set = set(random.sample(for_eval_set, len(val)))
/tmp/ipykernel_2140424/3740189284.py:42: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  test_set = set(random.sample(for_test_set, int(len(for_test_set)*0.2)))
 22%|██▏       | 2/9 [03:07<11:09, 95.70s/